# LSB Steganography and Computational Complexity
**Nombre:** Lucía Cantos Burgos 
**Asignatura:** Cryptography — Theme 2  
**Práctica:** LSB Steganography Lab


In [1]:
# Imports y configuración
from PIL import Image
import numpy as np
import os
import random

# Configuración base del lab
FOLDER = "dataset_images"
NUM_IMAGES = 100
W, H = 200, 200

# Cambia esto por tu nombre
FLAG = "FLAG{LUCIACANTOSBURGOS}"

def create_noise_image(filename, width=200, height=200):
    """
    Crea una imagen RGB de ruido aleatorio y la guarda en PNG.
    """
    arr = np.random.randint(0, 256, size=(height, width, 3), dtype=np.uint8)
    img = Image.fromarray(arr, mode="RGB")
    img.save(filename, format="PNG")

def text_to_binary(message: str) -> str:
    """
    Convierte un string a bits ASCII (8 bits por carácter).
    """
    return "".join(format(ord(ch), "08b") for ch in message)


END_DELIMITER = "1111111111111110"  # 16 bits



def hide_lsb(image_path: str, message: str):
    """
    Oculta un mensaje en una imagen PNG usando LSB y sobrescribe el archivo.
    """
    img = Image.open(image_path).convert("RGB")
    data = np.array(img, dtype=np.uint8)

    bits = text_to_binary(message) + END_DELIMITER
    flat = data.reshape(-1)  # aplana todos los canales en un vector

    if len(bits) > len(flat):
        raise ValueError("El mensaje es demasiado largo para la capacidad de la imagen.")

    # Inserción LSB
    for i, b in enumerate(bits):
        flat[i] = (flat[i] & 0xFE) | int(b)

    # Reconstruimos y guardamos
    stego = flat.reshape(data.shape)
    Image.fromarray(stego, mode="RGB").save(image_path, format="PNG")


def generate_dataset(folder: str, num_images: int, secret_message: str, width=200, height=200):
    os.makedirs(folder, exist_ok=True)

    filenames = []
    for i in range(num_images):
        fname = os.path.join(folder, f"img_{i:03d}.png")
        create_noise_image(fname, width=width, height=height)
        filenames.append(fname)

    secret_file = random.choice(filenames)
    hide_lsb(secret_file, secret_message)



generate_dataset(FOLDER, NUM_IMAGES, FLAG, width=W, height=H)
print(f"Dataset creado: {NUM_IMAGES} imágenes en '{FOLDER}/'")
print("Una de ellas contiene un mensaje oculto. Good luck!")



Dataset creado: 100 imágenes en 'dataset_images/'
Una de ellas contiene un mensaje oculto. Good luck!


## Explicación del generador

El generador crea un dataset de **N imágenes PNG** de ruido aleatorio (RGB) de tamaño **W×H**.  
Después selecciona **una imagen al azar** y oculta un mensaje `FLAG{...}` usando **LSB steganography**:

- Cada píxel tiene 3 canales (R,G,B), y se utiliza el bit menos significativo (LSB) de cada canal.
- El mensaje se convierte a binario ASCII (8 bits por carácter).
- Se añade un delimitador final (`1111111111111110`) para detectar el fin del mensaje.
- Para insertar cada bit se aplica:
  - limpiar LSB: `value & 0xFE`
  - insertar bit: `value | bit`

No se muestra qué imagen contiene el mensaje para simular un escenario forense real.


In [2]:
def extract_lsb(image_path: str, num_bits: int | None = None) -> str:
    """
    Extrae bits LSB de una imagen.
    
    Args:
        image_path: ruta del PNG
        num_bits: número de bits a extraer (None = todos)

    Returns:
        bitstring, por ejemplo "010101..."
    """
    img = Image.open(image_path).convert("RGB")
    data = np.array(img, dtype=np.uint8)

    # Aplanamos canales: [R,G,B,R,G,B,...]
    flat = data.reshape(-1)

    if num_bits is None:
        relevant = flat
    else:
        relevant = flat[:num_bits]

    # LSB de cada valor (0 o 1) -> lo convertimos a string
    bits = (relevant & 1).astype(np.uint8)
    return "".join(bits.astype(str))


def binary_to_text(bits: str, end_delimiter: str = END_DELIMITER) -> str:
    """
    Convierte bitstring a texto ASCII y corta al encontrar delimitador.
    """
    # Cortamos en el primer delimitador si aparece
    end_idx = bits.find(end_delimiter)
    if end_idx != -1:
        bits = bits[:end_idx]

    chars = []
    # Convertimos byte a byte
    for i in range(0, len(bits) - 7, 8):
        byte = bits[i:i+8]
        chars.append(chr(int(byte, 2)))
    return "".join(chars)

def search_brute_force(folder: str, header: str = HEADER):
    """
    Busca el mensaje con enfoque brute force.
    Devuelve (filename, message) si lo encuentra, o (None, None).
    """
    files = sorted([f for f in os.listdir(folder) if f.lower().endswith(".png")])

    for fname in files:
        path = os.path.join(folder, fname)

        bits = extract_lsb(path, num_bits=None)      # extrae TODO
        text = binary_to_text(bits)                  # reconstruye

        if header in text:
            return fname, text

    return None, None

def search_optimized(folder: str, header: str = HEADER):
    """
    Búsqueda optimizada: primero verifica header, luego extrae completo solo si coincide.
    Devuelve (filename, message) o (None, None).
    """
    files = sorted([f for f in os.listdir(folder) if f.lower().endswith(".png")])

    header_bits_len = len(header) * 8  # 5 chars * 8 = 40 bits

    for fname in files:
        path = os.path.join(folder, fname)

        # 1) Extraer solo lo mínimo para comprobar header
        bits_prefix = extract_lsb(path, num_bits=header_bits_len + len(END_DELIMITER))
        # Convertimos solo lo necesario a texto
        # (en realidad con 40 bits basta, pero añadir el delimitador aquí no estorba)
        text_prefix = binary_to_text(bits_prefix, end_delimiter=END_DELIMITER)

        if text_prefix.startswith(header):
            # 2) Ahora sí: extraemos todo para reconstruir mensaje completo
            bits_all = extract_lsb(path, num_bits=None)
            text_full = binary_to_text(bits_all, end_delimiter=END_DELIMITER)
            return fname, text_full

    return None, None


NameError: name 'HEADER' is not defined

## Explicación del detector (Parte 2)

El detector analiza todas las imágenes del dataset y extrae los bits LSB para reconstruir el mensaje.

Se implementan dos estrategias:

### 1) Brute force
Para cada imagen:
- se extraen todos los bits LSB,
- se reconstruye el texto completo (ASCII),
- se comprueba si aparece el patrón `"FLAG{"`.

Complejidad aproximada: **O(N·W·H)**.

### 2) Optimizado (early termination)
Para cada imagen:
- solo se extraen los primeros bits necesarios para reconstruir el header `"FLAG{"` (k bytes),
- si coincide, entonces se extrae el mensaje completo,
- si no coincide, se descarta inmediatamente.

Complejidad aproximada: **O(N·k)** con `k << W·H`.


In [ ]:
END_DELIMITER = "1111111111111110"
HEADER = "FLAG{"

def create_noise_image(filename, width, height):
    arr = np.random.randint(0, 256, size=(height, width, 3), dtype=np.uint8)
    Image.fromarray(arr, mode="RGB").save(filename, format="PNG")

def text_to_binary(message: str) -> str:
    return "".join(format(ord(ch), "08b") for ch in message)

def hide_lsb(image_path: str, message: str):
    img = Image.open(image_path).convert("RGB")
    data = np.array(img, dtype=np.uint8)
    flat = data.reshape(-1)

    bits = text_to_binary(message) + END_DELIMITER
    if len(bits) > len(flat):
        raise ValueError("Mensaje demasiado largo para esta imagen.")

    for i, b in enumerate(bits):
        flat[i] = (flat[i] & 0xFE) | int(b)

    Image.fromarray(flat.reshape(data.shape), mode="RGB").save(image_path, format="PNG")

def generate_dataset(folder: str, N: int, W: int, H: int, flag: str):
    os.makedirs(folder, exist_ok=True)

    # Limpieza opcional: borrar PNG anteriores (para que N sea exacto)
    for f in os.listdir(folder):
        if f.lower().endswith(".png"):
            os.remove(os.path.join(folder, f))

    files = []
    for i in range(N):
        fname = os.path.join(folder, f"img_{i:03d}.png")
        create_noise_image(fname, W, H)
        files.append(fname)

    secret = random.choice(files)
    hide_lsb(secret, flag)


def extract_lsb(image_path: str, num_bits: int | None = None) -> str:
    img = Image.open(image_path).convert("RGB")
    data = np.array(img, dtype=np.uint8)
    flat = data.reshape(-1)

    if num_bits is None:
        relevant = flat
    else:
        relevant = flat[:num_bits]

    bits = (relevant & 1).astype(np.uint8)
    return "".join(bits.astype(str))

def binary_to_text(bits: str, end_delimiter: str = END_DELIMITER) -> str:
    end_idx = bits.find(end_delimiter)
    if end_idx != -1:
        bits = bits[:end_idx]

    out = []
    for i in range(0, len(bits) - 7, 8):
        out.append(chr(int(bits[i:i+8], 2)))
    return "".join(out)

def search_brute_force(folder: str, header: str = HEADER):
    files = sorted([f for f in os.listdir(folder) if f.lower().endswith(".png")])
    for fname in files:
        path = os.path.join(folder, fname)
        bits = extract_lsb(path, None)
        text = binary_to_text(bits)
        if header in text:
            return fname, text
    return None, None

def search_optimized(folder: str, header: str = HEADER):
    files = sorted([f for f in os.listdir(folder) if f.lower().endswith(".png")])
    header_bits = len(header) * 8  # 40 bits

    for fname in files:
        path = os.path.join(folder, fname)

        bits_prefix = extract_lsb(path, header_bits)
        text_prefix = binary_to_text(bits_prefix, end_delimiter="")  # sin delimiter para prefix

        if text_prefix.startswith(header):
            bits_all = extract_lsb(path, None)
            text_full = binary_to_text(bits_all, end_delimiter=END_DELIMITER)
            return fname, text_full

    return None, None

def time_once(fn, *args, **kwargs):
    t0 = time.perf_counter()
    result = fn(*args, **kwargs)
    t1 = time.perf_counter()
    return (t1 - t0), result

def bench_detector(folder, reps=3):
    # Brute force
    brute_times = []
    brute_res = None
    for _ in range(reps):
        dt, res = time_once(search_brute_force, folder)
        brute_times.append(dt)
        brute_res = res
    brute_best = min(brute_times)

    # Optimizado
    opt_times = []
    opt_res = None
    for _ in range(reps):
        dt, res = time_once(search_optimized, folder)
        opt_times.append(dt)
        opt_res = res
    opt_best = min(opt_times)

    return brute_best, opt_best, brute_res, opt_res
FLAG = "FLAG{LUCIACANTOSBURGOS}"
BASE = "bench_datasets"

configs = [
    (10, 200, 200),
    (50, 200, 200),
    (100, 200, 200),
    (100, 500, 500),
    (100, 1000, 1000),
]

results = []

for (N, W, H) in configs:
    folder = os.path.join(BASE, f"N{N}_W{W}_H{H}")
    print(f"\n== Generando dataset: N={N}, {W}x{H} ==")
    generate_dataset(folder, N, W, H, FLAG)

    print("   Midiendo brute vs optimizado...")
    brute_t, opt_t, brute_res, opt_res = bench_detector(folder, reps=3)

    results.append({
        "N": N,
        "WxH": f"{W}x{H}",
        "BruteForce_sec": brute_t,
        "Optimized_sec": opt_t,
        "Speedup": (brute_t / opt_t) if opt_t > 0 else None
    })

results

for r in results:
    print(f"{r['N']:>4} | {r['WxH']:<9} | brute={r['BruteForce_sec']:.4f}s | opt={r['Optimized_sec']:.4f}s | x{r['Speedup']:.1f}")



## Timing Results

| N (images) | W x H | Brute Force (sec) | Optimized (sec) | Speedup |
|------------|-------|-------------------|-----------------|---------|
| 10  | 200x200   | 0.1077 | 0.0038 | ×28.6 |
| 50  | 200x200   | 1.0544 | 0.0209 | ×50.3 |
| 100 | 200x200   | 0.6987 | 0.0392 | ×17.8 |
| 100 | 500x500   | 12.2824 | 0.4382 | ×28.0 |
| 100 | 1000x1000 | 13.9528 | 1.7922 | ×7.8 |


## 4.2.1 Question 1: Theoretical Complexity

Sea:
- N = nº de imágenes
- W, H = dimensiones
- k = tamaño del header en bytes (p.ej. k=5 para `"FLAG{"`)

### (a) Complejidad brute force
Brute force extrae LSB de TODOS los píxeles (y 3 canales) para TODAS las imágenes:

\[
O(N \cdot W \cdot H)
\]

(más precisamente, \(O(N \cdot W \cdot H \cdot 3)\), pero el 3 es constante).

### (b) Complejidad optimizada (early termination)
Solo extrae lo necesario para comprobar el header (k bytes = 8k bits) por imagen:

\[
O(N \cdot k)
\]

y cuando encuentra la correcta, paga un coste extra de extracción completa una vez:
\[
O(N \cdot k + W\cdot H)
\]
(ese segundo término ocurre solo para 1 imagen).

### (c) Si W=H=1000 y k=5, ¿cuántas veces más rápido?
En brute force, bits potenciales por imagen ≈ \(W\cdot H\cdot 3\).
En optimizado, bits leídos ≈ \(8k = 40\).

Factor teórico:
\[
\frac{W\cdot H\cdot 3}{8k}
= \frac{1000\cdot1000\cdot 3}{40}
= \frac{3,000,000}{40}
= 75,000
\]

→ Teóricamente, el optimizado puede ser ~**75.000×** más rápido en trabajo de extracción de bits.
(En la práctica, el speedup real puede ser menor por coste de lectura/decodificación PNG.)


## 4.2.2 Question 2: Bottlenecks

### (a) ¿Qué tarda más: I/O (leer) o CPU (calcular bits)?


Los experimentos muestran que el principal cuello de botella es la lectura y decodificación de la imagen (I/O).
En imágenes grandes, este coste es aproximadamente 15 veces mayor que el coste de extraer los bits LSB una vez la imagen está en memoria.

### (b) ¿Cómo verificarlo empíricamente?
Se midió por separado el tiempo de:

abrir y convertir una imagen PNG a un array NumPy

extraer los bits LSB desde un array ya en memoria

Comparando ambos tiempos se observa que el coste de I/O domina claramente.

### (c) Si el cuello es I/O, ¿ayuda un procesador más rápido?
No significativamente.
Dado que el cuello de botella es I/O y decodificación, mejorar la CPU tiene un impacto limitado.
Sería más efectivo mejorar el subsistema de almacenamiento o paralelizar la lectura.


Aunque el análisis teórico predice una mejora de varios órdenes de magnitud, los resultados experimentales muestran que el speedup real está limitado por el coste de I/O y decodificación de imágenes, lo que evidencia la diferencia entre complejidad algorítmica y rendimiento real en sistemas prácticos.


## 4.2.3 Question 3: Scalability

### (a) Si tuviera 1 millón de imágenes, ¿cuánto tardaría el optimizado?
Usamos el tiempo medio por imagen del optimizado (medido).
Si para N=100 el optimizado tardó `T_100`, entonces aprox:

\[
t_{img} \approx \frac{T_{100}}{100}
\quad\Rightarrow\quad
T_{1,000,000} \approx 1,000,000 \cdot t_{img}
\]

(Esto asume comportamiento lineal con N, que es lo esperable.)

### (b) ¿Cómo usar multiprocessing?
Dividir la lista de imágenes en P bloques y que cada proceso analice un bloque.
Cuando uno encuentra el mensaje, se puede cancelar el resto (en la práctica, con colas/eventos).

### (c) Complejidad con P procesadores
Idealmente:
\[
O\left(\frac{N\cdot k}{P}\right)
\]
pero en la práctica hay overhead (arranque de procesos, lectura de disco, contención I/O).


## 4.2.4 Question 4: Steganography Security

### (a) Si el atacante NO usa un header predecible como "FLAG{", ¿se puede automatizar la búsqueda?
Se vuelve muchísimo más difícil.
Con header, la detección es un problema de “pattern matching” rápido.
Sin header, no puedes distinguir fácilmente entre:
- bits aleatorios del LSB (ruido natural o aleatorio)
- bits de un mensaje cifrado (que también parece aleatorio)

La búsqueda automática se puede hacer, pero ya no es por firma ("FLAG{"), sino por **detección estadística** (esteganoanálisis), y tendrá falsos positivos/negativos.

### (b) ¿Cómo distinguir “random noise” vs “encrypted message”?
Un mensaje cifrado bien hecho produce bits ~uniformes (50/50), igual que el ruido.
Así que por distribución simple de 0/1 puede ser indistinguible.

Estrategias:
- buscar estructura (delimitadores, longitudes, redundancia) → si existen
- tests estadísticos más finos (chi-cuadrado, RS steganalysis, correlaciones locales)
- comparar con el modelo esperado de una imagen “natural” (no ruido puro)

En este lab, como las imágenes son *ruido aleatorio*, la detección estadística es todavía más difícil; por eso el header es clave.

### (c) ¿Qué puede hacer el atacante para dificultar detección?
- Insertar bits en posiciones pseudoaleatorias con una clave (no secuencial)
- Cifrar + comprimir el mensaje antes de insertar (menos patrones)
- Usar LSB matching (no solo setear el bit, sino ajustar ±1 aleatoriamente)
- Repartir el payload en menos píxeles o en canales específicos
- Cambiar de dominio (p.ej. JPEG/DCT) en lugar de LSB directo en PNG
- Usar técnicas adaptativas (inserta más en zonas con textura donde se nota menos)


In [ ]:
FOLDER = "dataset_images"

def load_rgb_array(path: str) -> np.ndarray:
    img = Image.open(path).convert("RGB")
    return np.array(img, dtype=np.uint8)

def flat_channels(arr: np.ndarray) -> np.ndarray:
    # devuelve vector [R,G,B,R,G,B,...]
    return arr.reshape(-1)

def lsb_bits(flat: np.ndarray) -> np.ndarray:
    return (flat & 1).astype(np.uint8)

def chi_square_lsb_01(flat: np.ndarray) -> float:
    bits = lsb_bits(flat)
    n = bits.size
    o1 = int(bits.sum())
    o0 = n - o1
    e = n / 2.0
    # evita división por 0 (n>0 siempre)
    chi2 = ((o0 - e)**2)/e + ((o1 - e)**2)/e
    return float(chi2)

def lsb_bias(flat: np.ndarray) -> float:
    bits = lsb_bits(flat)
    return float(bits.mean() - 0.5)  # desviación respecto 0.5

def pov_score_channel(values_1d: np.ndarray) -> float:
    hist = np.bincount(values_1d, minlength=256).astype(np.int64)
    diff_sum = 0
    total = 0
    for k in range(128):
        a = hist[2*k]
        b = hist[2*k + 1]
        diff_sum += abs(int(a - b))
        total += int(a + b)
    if total == 0:
        return 0.0
    # 0 -> muy diferente, 1 -> muy igualado
    return float(1.0 - (diff_sum / total))

def pov_score_rgb(arr: np.ndarray) -> float:
    r = arr[:,:,0].reshape(-1)
    g = arr[:,:,1].reshape(-1)
    b = arr[:,:,2].reshape(-1)
    return float((pov_score_channel(r) + pov_score_channel(g) + pov_score_channel(b)) / 3.0)


def block_smoothness(block: np.ndarray) -> int:
    # block shape: (bh, bw) uint8
    # suma de diferencias absolutas horizontales + verticales
    block = block.astype(np.int16)
    dh = np.abs(block[:, 1:] - block[:, :-1]).sum()
    dv = np.abs(block[1:, :] - block[:-1, :]).sum()
    return int(dh + dv)

def flip_lsb(block: np.ndarray) -> np.ndarray:
    return (block ^ 1).astype(np.uint8)  # togglear LSB

def rs_proxy_score(arr: np.ndarray, block_size: int = 8) -> float:
    """
    Score: fracción de bloques donde flipping LSB aumenta smoothness de forma consistente.
    Valores más altos -> más sospechoso.
    """
    # trabajamos en luminancia simple para simplificar (promedio canales)
    Y = arr.mean(axis=2).astype(np.uint8)
    H, W = Y.shape
    bs = block_size
    h_blocks = H // bs
    w_blocks = W // bs

    if h_blocks == 0 or w_blocks == 0:
        return 0.0

    inc = 0
    total = 0

    for i in range(h_blocks):
        for j in range(w_blocks):
            block = Y[i*bs:(i+1)*bs, j*bs:(j+1)*bs]
            s0 = block_smoothness(block)
            s1 = block_smoothness(flip_lsb(block))
            if s1 > s0:
                inc += 1
            total += 1

    return float(inc / total)


def zscore(x: np.ndarray) -> np.ndarray:
    mu = x.mean()
    sigma = x.std()
    if sigma == 0:
        return np.zeros_like(x)
    return (x - mu) / sigma

def analyze_folder(folder: str):
    files = sorted([f for f in os.listdir(folder) if f.lower().endswith(".png")])

    chi2_list = []
    bias_list = []
    pov_list = []
    rs_list = []

    for fname in files:
        path = os.path.join(folder, fname)
        arr = load_rgb_array(path)
        flat = flat_channels(arr)

        chi2_list.append(chi_square_lsb_01(flat))
        bias_list.append(abs(lsb_bias(flat)))
        pov_list.append(pov_score_rgb(arr))
        rs_list.append(rs_proxy_score(arr, block_size=8))

    chi2 = np.array(chi2_list, dtype=float)
    bias = np.array(bias_list, dtype=float)
    pov  = np.array(pov_list, dtype=float)
    rs   = np.array(rs_list, dtype=float)

    # Normalizamos para combinar (z-score)
    z_chi2 = zscore(chi2)
    z_bias = zscore(bias)
    z_pov  = zscore(pov)
    z_rs   = zscore(rs)

    # Score combinado (ajústalo si quieres: aquí damos más peso a PoV y RS)
    combined = 0.2*z_chi2 + 0.1*z_bias + 0.4*z_pov + 0.3*z_rs

    # Ranking (más alto = más sospechoso)
    order = np.argsort(-combined)

    ranked = []
    for idx in order:
        ranked.append({
            "file": files[idx],
            "combined": float(combined[idx]),
            "chi2": float(chi2[idx]),
            "bias": float(bias[idx]),
            "pov": float(pov[idx]),
            "rs": float(rs[idx]),
        })
    return ranked

ranked = analyze_folder(FOLDER)
ranked[:10]

print("Top 10 sospechosas (mayor score combinado):\n")
for r in ranked[:10]:
    print(f"{r['file']:<12} combined={r['combined']:+.3f} | chi2={r['chi2']:.2f} bias={r['bias']:.5f} pov={r['pov']:.5f} rs={r['rs']:.5f}")



## CONCLUSIONES

 
En este apartado se analizan los resultados obtenidos al aplicar diferentes técnicas de esteganoanálisis estadístico sobre el conjunto de imágenes, sin conocimiento previo del formato del mensaje ni de un header identificable. El objetivo es evaluar si es posible **priorizar imágenes sospechosas** basándose únicamente en anomalías estadísticas en los bits LSB.

---

### 1. Visión general del ranking

Se ha construido un ranking de imágenes sospechosas combinando **cuatro métricas estadísticas independientes**:

- Test **χ²** sobre la distribución de bits LSB (0/1)
- **Bias** LSB (desviación respecto al 50%)
- **Pair-of-Values (PoV)** sobre pares (2k, 2k+1)
- **RS proxy** basado en análisis por bloques

El hecho de que ninguna métrica individual domine el ranking y, aun así, aparezcan imágenes claramente destacadas, indica que el enfoque combinado es **robusto y no dependiente de una sola prueba**.

La imagen más sospechosa es:

- **img_042.png**, con un score combinado de **+1.866**

Además, el ranking presenta una **caída progresiva del score**, lo que sugiere que no se trata de ruido aleatorio sino de una estructura estadística coherente.

---

### 2. Análisis por métricas

#### 2.1 Chi-Square (LSB 0/1)

Observaciones:
- Los valores de χ² oscilan aproximadamente entre **0.36 y 5.96**.
- Algunas imágenes (como *img_042.png* y *img_076.png*) muestran valores de χ² sensiblemente más altos.

Interpretación:
- Dado que las imágenes base son ruido aleatorio, no se esperan desviaciones extremas.
- Sin embargo, ciertas imágenes presentan una desviación mayor que el promedio, lo que puede indicar una alteración sistemática de los bits LSB.

Conclusión:
> El test χ² por sí solo no es concluyente, pero aporta señal útil cuando se combina con otras métricas.

---

#### 2.2 Bias LSB (desviación del 50%)

Observaciones:
- Los valores de bias son pequeños (≈ 0.001 – 0.0035), como era esperable.
- Las imágenes mejor rankeadas tienden a presentar un bias ligeramente superior.

Interpretación:
- El mensaje oculto es pequeño en comparación con el número total de píxeles, por lo que la desviación global es reducida.
- Aun así, el bias permite detectar **desviaciones sistemáticas**, no puramente aleatorias.

Conclusión:
> El bias global es un detector débil de forma aislada, pero consistente cuando se analiza de forma comparativa.

---

#### 2.3 Pair-of-Values (PoV)

Observaciones:
- Los valores PoV se concentran en un rango alto (~0.955–0.959).
- Las imágenes con mayor score combinado tienden a presentar valores PoV ligeramente superiores.

Interpretación:
- El reemplazo de LSB tiende a igualar las frecuencias de valores pares (2k) e impares vecinos (2k+1).
- Incluso en imágenes de ruido, la inserción secuencial introduce una igualación adicional detectable.

Conclusión:
> PoV resulta ser una de las métricas más discriminativas del conjunto, especialmente adecuada para detectar LSB replacement.

---

#### 2.4 RS Proxy (análisis por bloques)

Observaciones:
- Los valores RS se sitúan aproximadamente entre **0.49 y 0.55**.
- Las imágenes sospechosas tienden a valores ligeramente más altos (>0.52).

Interpretación:
- El flipping de LSB introduce perturbaciones locales que afectan a la regularidad de bloques.
- Este análisis captura **anomalías estructurales locales**, complementando métricas globales.

Conclusión:
> El RS proxy aporta información local relevante que no es capturada por tests globales como χ² o bias.

---

### 3. Caso destacado: img_042.png

La imagen **img_042.png** destaca de forma consistente en todas las métricas:

- Mayor valor de χ²
- Mayor bias LSB
- PoV elevado
- RS proxy coherente con inserción LSB

Esto indica que la imagen no sobresale por una única prueba aislada, sino por la **acumulación coherente de evidencias estadísticas**, lo cual es característico de un enfoque de esteganoanálisis sólido.

---

### 4. Relación con el escenario “Detection Without Header”

Estos resultados reflejan un escenario realista de análisis forense:

- Sin un header conocido, no es posible identificar el mensaje de forma determinista.
- La detección se convierte en un problema de **clasificación probabilística**, no de decodificación.
- El objetivo es **priorizar imágenes sospechosas**, reduciendo el espacio de búsqueda.

> En ausencia de una firma conocida, la detección automática se convierte en un problema de clasificación estadística, donde múltiples tests débiles combinados producen un detector robusto.

---

### 5. Limitaciones del enfoque

- Las imágenes analizadas son de **ruido aleatorio**, no imágenes naturales.
- En ruido puro, las distribuciones LSB ya son cercanas a uniformes.
- Esto reduce la separabilidad estadística entre imágenes con y sin mensaje.
- En escenarios reales con imágenes naturales, estas técnicas suelen ser más efectivas.

---

### 6. Conclusión final

Los resultados muestran que, incluso sin conocer el formato del mensaje ni un header predecible, es posible identificar imágenes sospechosas mediante esteganoanálisis estadístico.  
Aunque ninguna métrica individual es suficiente por sí sola, la combinación de pruebas globales (χ², bias), histogramales (PoV) y locales (RS proxy) permite construir un ranking coherente que prioriza imágenes con mayor probabilidad de contener información oculta.  

Este enfoque refleja fielmente el análisis forense real, donde la detección es probabilística y se basa en la agregación de múltiples evidencias estadísticas.